In [22]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import dask.dataframe as dd
from toolbox.data_aggregation import *
from toolbox.linear_reg import *
import datetime
from datetime import timedelta, date
import numpy as np

In [23]:
X_train = pd.read_csv('data/X_train_2017.csv')
y_train = np.genfromtxt('data/y_train_2017.csv', delimiter=",")

# Modify target to fit official evaluation

In [24]:
y_train = np.multiply(y_train, 1 + 0.118 * X_train['item_perishable'].values.reshape([-1, 1]))

# Shuffle and split
date, store, item are all randomized

In [25]:
from sklearn.model_selection import train_test_split
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [38]:
y_test_pred = np.zeros(y_test_s.shape)

In [92]:
xgb_reg_list = [XGBRegressor() for i in range(16)]

In [58]:
from xgboost import XGBRegressor

for i in range(16):
    xgb_reg_list[i].fit(X_train_s, y_train_s[:, i])
    y_test_pred[:, i] = xgb_reg_list[i].predict(X_test_s)


In [64]:
y_test_truth_wow = np.multiply(y_test_s, 1/(1 + 0.118 * X_test_s['item_perishable'].values.reshape([-1, 1])))
y_test_pred_wow = np.multiply(y_test_pred, 1/(1 + 0.118 * X_test_s['item_perishable'].values.reshape([-1, 1])))

In [89]:
from sklearn.metrics import mean_squared_error
weight = 1 + 0.25 * X_test_s['item_perishable'].values.reshape([1, -1])
result_test = mean_squared_error(y_test_truth_wow.reshape([1, -1]), y_test_pred_wow.reshape([1, -1]),
                                 weight.reshape([1, -1]).repeat(16, axis=0).reshape([1, -1]))

In [91]:
print("Test set evaluation: ", np.sqrt(result_test))

0.62519352436217157